In [ ]:
pip install bottle

In [ ]:
from bottle import route, run

@route('/hola')
def hello():
    return "Hola mundo!"
run(host='0.0.0.0',port=5000,debug=True)

**route() es un decorador que conecta una función a una URL, en este caso conectamos la ruta a la funcion hello(). Esto es llamado una ruta y es uno de los conceptos mas importantes de este framework. Se pueden definir tantas rutas como sean necesarias y cuando desde un navegador solicite una URL, la funcion asociada a esa ruta sera ejecutada y respondera al navegador.**

**La función run() en las ultimas lineas inicia el servidor web de desarrollo, este se ejecuta en local en el puerto 8080(puede definirse el puerto), y responde a las solicitudes hasta que presionamos ctrl+c. Podemos cambiar el servidor, esto en caso de llevar a "produccion" nuestra aplicación, pero para desarrollo con las opciones default nos es suficiente, no necesita instalarse y es lo mas sencillo para probar nuestra aplicación.**

## La apicacion default

**Podemos observar en el ejemplo que usamos el decorador route() para definir las rutas, esto añade las rutas a nuestra aplicacion default, una instancia qeu se crea la primera vez que usamos el decorador route(), pero para seguir un enfoque mas orientado a objetos, creamos primero la instancia de la aplicacion y añadimos estas rutas a nuestra instacia explicitamente**

In [ ]:
from bottle import Bottle, run

app = Bottle()

@app.route('/hola')
def hello():
    return "Hola mundo!"

run(app, host='0.0.0.0',port=5000,debug=True)

## Añadimos mas de una ruta
**Podemos añadir mas de una ruta a una función, ademas podemos añadir "comodines"(wildcards) a la ruta para hacerla mas dinamica. Ademas veremos en acción el modulo template, donde nuestra función regresara un valor y sera reemplazado en nuestro HTML**

In [ ]:
from bottle import Bottle, run, template

app = Bottle()

@app.route('/')
@app.route('/hola /<name>')
def hello(name='Extraño'):
    return template('Hola {{name}}, como estas?',name=name)

run(app, host='0.0.0.0',port=5000,debug=True)

## Un ejemplo de POST y GET
**Podemos ver un ejemplo de como manejar los diferentes verbos o metodos http con el ejemplo de un login sencillo**

In [12]:
from bottle import Bottle, run, template, request, redirect

app = Bottle()

@app.route('/')
@app.route('/hola /<name>')
def hello(name='Extraño'):
    return template('Hola {{name}}, como estas?',name=name)

@app.route('/loged/<name>')
def loged(name='Extraño'):
    return template('''You're in! {{name}} ''',name=name)

def checkLogin(username, password):
    isLogged = False
    if username == 'cesar' and password == '123':
        isLogged = True
    return isLogged
@app.route('/login',method='GET')
def login():
    isFailed = '0' if request.query.Failed == '' else request.query.Failed
    
    formString = '''<form action="/login"method="post">
            Username: <input name ="username" type="text" />
            Password: <input name ="password" type="password" />
            <input value="Login" type="submit" />
            </form>
    '''
    
    if int(isFailed) == 1:
        return '<p><strong>Login failed!</p>'+formString
    else:
        return formString

@app.post('/login') #or @route('/login',method ='POST')
def do_login():
    username = request.forms.get('username')
    password = request.forms.get('password')
    if checkLogin(username,password):
        redirect(f"/loged/{username}")
    else:
        redirect("/login?Failed=1")
        

run(app, host='0.0.0.0',port=5000,debug=True)

Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:5000/
Hit Ctrl-C to quit.

10.0.2.2 - - [15/Mar/2019 10:54:20] "GET /login HTTP/1.1" 200 236
10.0.2.2 - - [15/Mar/2019 10:54:25] "POST /login HTTP/1.1" 303 0
10.0.2.2 - - [15/Mar/2019 10:54:25] "GET /login?Failed=1 HTTP/1.1" 200 264
10.0.2.2 - - [15/Mar/2019 10:54:31] "POST /login HTTP/1.1" 303 0
10.0.2.2 - - [15/Mar/2019 10:54:31] "GET /loged/cesar HTTP/1.1" 200 17
